In [26]:
import pandas as pd 
import numpy as np 
import os 
from config.path_config import ROOT_DIR
from sklearn.model_selection import train_test_split,StratifiedKFold
from sklearn.metrics import silhouette_samples, silhouette_score,davies_bouldin_score
from sklearn.cluster import KMeans
from sklearn import metrics
from lightgbm import LGBMClassifier
TARGET = "SeriousDlqin2yrs"
TRAINING_PATH = os.path.join(ROOT_DIR,"data","processed","processed_training_forth_exp_tuned.csv")

In [13]:
training_data = pd.read_csv(TRAINING_PATH)

In [14]:
#PERFORM ANOTHER FEATURE ENGINEERING -> KMeans for number of days late 


feature_number_of_lates = ['NumberOfTimes90DaysLate','NumberOfTime60-89DaysPastDueNotWorse','NumberOfTime30-59DaysPastDueNotWorse']
data=training_data[feature_number_of_lates]
clusterer = KMeans(n_clusters=4,n_init=10, random_state=10)
cluster_labels = clusterer.fit_predict(data)
# silhouette_avg = silhouette_score(data, cluster_labels)
# print("For n_clusters =", 3,
#       "The average silhouette_score is :", silhouette_avg)

In [15]:
training_data['late_labels_cluster'] = cluster_labels

In [31]:
#split into X and Y 
X = training_data.drop([TARGET,'late_labels_cluster'],axis=1)
Y = training_data[TARGET]

In [32]:
fold = StratifiedKFold(5)
scoring_dict = {"auc":[]}
for train_idx, test_idx in fold.split(X, Y):
    train_x, val_x = X.iloc[train_idx], X.iloc[test_idx]
    train_y, val_y = Y.iloc[train_idx], Y.iloc[test_idx]
    # train model

    model = LGBMClassifier()
    model.fit(train_x, train_y)

    y_pred = model.predict_proba(val_x)
    # calculate metrics
    scoring_dict["auc"].append(metrics.roc_auc_score(val_y, y_pred[:, 1]))

In [33]:
np.mean(scoring_dict['auc'])

0.8645873614338461

In [34]:
#split into X and Y 
X = training_data.drop([TARGET],axis=1)
Y = training_data[TARGET]

In [35]:
fold = StratifiedKFold(5)
scoring_dict = {"auc":[]}
for train_idx, test_idx in fold.split(X, Y):
    train_x, val_x = X.iloc[train_idx], X.iloc[test_idx]
    train_y, val_y = Y.iloc[train_idx], Y.iloc[test_idx]
    # train model

    model = LGBMClassifier()
    model.fit(train_x, train_y)

    y_pred = model.predict_proba(val_x)
    # calculate metrics
    scoring_dict["auc"].append(metrics.roc_auc_score(val_y, y_pred[:, 1]))

In [36]:
np.mean(scoring_dict['auc'])

0.8648491661593176